In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
#import library
import pandas as pd
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [ ]:
train_lab = pd.read_csv('train_v2.csv')
train_lab.head()

In [ ]:
sub = pd.read_csv('sample_submission_v2.csv')
sub.head()

In [ ]:
image_number =10
img = io.imread('https://storage.googleapis.com/kaggle-competitions/kaggle/6322/media/habitation1.jpg'.format(image_number))

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
#check value_counts()
train_lab.tags.value_counts()

In [ ]:
unique = set()
def append_labels(tags):
    for tag in tags.split():
        unique.add(tag)

train_labs = train_lab.copy()
train_labs['tags'].apply(append_labels)
unique = list(unique)
print(unique)

In [ ]:
# adding '.jpg' extension to 'image_name'
train_labs['image_name'] = train_labs['image_name'].apply(lambda x: '{}.jpg'.format(x)) 
train_labs.head()

In [ ]:
y_col = list(train_labs.columns[2:]) # storing the tags column names as a variable

# initializing an image generator with some data augumentation
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

# loading images from dataframe
X = image_gen.flow_from_dataframe(dataframe=train_labs, \
        directory='https://storage.googleapis.com/kaggle-competitions/kaggle/6322/media/habitation1.jpg', x_col='image_name', y_col=y_col, \
       target_size=(64, 64), class_mode='raw', seed=1, batch_size=64, validate_filenames=False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD

In [ ]:
def fbeta(ytrue , ypred, beta=2, epsilon=1e-4):
    beta_squarred = beta**2

    ytrue = tf.cast(ytrue, tf.float32)
    ypred = tf.cast(tf.greater(tf.cast(ypred, tf.float32), tf.constant(0.5)), tf.float32)
        
    tp = tf.reduce_sum(ytrue * ypred, axis=1)
    fp = tf.reduce_sum(ypred, axis=1) - tp
    fn = tf.reduce_sum(ytrue, axis=1) - tp
    
    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fb = (1+beta_squarred)*precision*recall / (beta_squarred*precision + recall + epsilon)
    return fb

In [ ]:
def multi_label_acc(ytrue , ypred, epsilon=1e-4):
    
    ytrue = tf.cast(ytrue, tf.float32)
    ypred = tf.cast(tf.greater(tf.cast(ypred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(ytrue * ypred, axis=1)
    fp = tf.reduce_sum(ypred, axis=1) - tp
    fn = tf.reduce_sum(ytrue, axis=1) - tp
    
    ytrue = tf.cast(ytrue, tf.bool)
    ypred = tf.cast(ypred, tf.bool)
    
    tn = tf.reduce_sum(tf.cast(tf.logical_not(ytrue), tf.float32) * tf.cast(tf.logical_not(ypred), tf.float32),\
                       axis=1)
    
    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [ ]:
def build_model():
    in_shape=(64, 64, 3)
    out_shape=17

    # load mannequin

    mannequin = VGG16(include_top=False, input_shape=in_shape)

    # mark loaded layers as not trainable

    for layer in mannequin.layers:
        layer.trainable = False

    # enable final vgg block to be trainable

    mannequin.get_layer("block5_conv1").trainable = True

    mannequin.get_layer("block5_conv2").trainable = True

    mannequin.get_layer("block5_conv3").trainable = True

    mannequin.get_layer("block5_pool").trainable = True

    # add new classifier layers

    flat1 = Flatten()(mannequin.layers[-1].output)

    class1 = Dense(128, activation="relu", kernel_initializer="he_uniform")(flat1)

    output = Dense(out_shape, activation="sigmoid")(class1)

    # outline new mannequin

    mannequin = Model(inputs=mannequin.inputs, outputs=output)

    # compile mannequin

    choose = SGD(learning_rate=0.01, momentum=0.9)

    mannequin.compile(optimizer=choose, loss="binary_crossentropy", metrics=[fbeta, multi_label_acc])
    
    return mannequin

In [ ]:
save_best_check_point = ModelCheckpoint(filepath='best_model.hdf5', monitor='val_fbeta',mode='max', save_best_only=True, save_weights_only=True)

In [ ]:
train_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,validation_split=0.2)

# generating the 80% training image data
train_gen = train_image_gen.flow_from_dataframe(dataframe=train_labs, \
        directory='https://storage.googleapis.com/kaggle-competitions/kaggle/6322/media/habitation1.jpg', x_col='image_name', y_col=y_col, \
       target_size=(64, 64), class_mode='raw', seed=0, batch_size=64, subset='training', validate_filenames=False)

# generating the 20% validation image data
val_gen = train_image_gen.flow_from_dataframe(dataframe=train_labs, \
        directory='https://storage.googleapis.com/kaggle-competitions/kaggle/6322/media/habitation1.jpg', x_col='image_name', y_col=y_col, \
       target_size=(64,64), class_mode='raw', seed=0, batch_size=64, subset='validation', validate_filenames=False)

In [ ]:
# setting step size for training and validation image data
step_train_size = int(np.ceil(train_gen.samples / train_gen.batch_size))
step_val_size = int(np.ceil(val_gen.samples / train_gen.batch_size))
step_train_size+step_val_size

In [ ]:
train_labs.head()

In [ ]:
label_list =[]

for tag_str in train_labs.tags.values:
    labels =tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

label_list

In [ ]:
for label in label_list:
    train_labs[label] =train_labs['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
# Display head
train_labs.head()

In [ ]:
import seaborn as sns
def make_cooccurence_matrix(labels):
    numeric_df = train_labs[labels]; 
    c_matrix = numeric_df.T.dot(numeric_df)
    sns.heatmap(c_matrix)
    return c_matrix
    
# Compute the co-ocurrence matrix
make_cooccurence_matrix(label_list)

In [ ]:
image_number =10
imgs = io.imread('https://storage.googleapis.com/kaggle-competitions/kaggle/6322/media/agg1.jpg'.format(image_number))
plt.imshow(img)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed

In [ ]:
def create_tag_mapping(labels_df):
    
    labels = set()
    for i in range(len(labels_df)):
        # convert spaced separated tags into an array of tags
        tags = labels_df['tags'][i].split(' ')
        # add tags to the set of known labels
        labels.update(tags)
    # convert set of labels to a list to list
    labels = list(labels)
    labels.sort()
 # dict that maps labels to integers, and the reverse
    labels_map = { labels [i]:i for i in range(len(labels))}
    inv_labels_map = {i: labels [i] for i in range(len(labels))}
    return labels_map, inv_labels_map



def create_file_mapping(train_data):
    mapping = dict()
    for i in range(len(labels_df)):
        name, tags = train_data['image_name'][i], labels_df['tags'][i]
        mapping[name] = tags.split(' ')
        
    return mapping


def one_hot_encode(tags,mapping):
    
    # create empty vector
    encoding = zeros(len(mapping), dtype='uint8')
    # mark 1 for each tag in the vector
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding




def load_dataset(path, file_mapping, tag_mapping):
    photos, targets = list(), list()
    # enumerate files in the directory
    for filename in listdir(folder):
        # load image
        photo = load_img(path + filename, target_size=(64,64))
        # convert to numpy array
        photo = img_to_array(photo, dtype='uint8')
        # get tags
        tags = file_mapping[filename[:-4]]
        # one hot encode tags
        target = one_hot_encode(tags, tag_mapping)
        # store
        photos.append(photo)
        targets.append(target)
    X = asarray(photos, dtype='uint8')
    y = asarray(targets, dtype='uint8')
    return X, y

In [ ]:
filename = 'train_v2.csv'
labels_df = pd.read_csv('train_v2.csv', encoding='latin1')

# create a mapping of tags to integers
mapping, inv_mapping = create_tag_mapping(labels_df)
print(len(mapping))
print(mapping)

# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(labels_df)

# create a mapping of filenames to tag lists
file_mapping = create_file_mapping(labels_df)

# load the jpeg images
folder = 'https://storage.googleapis.com/kaggle-competitions/kaggle/6322/media/habitation1.jpg'
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)

# save both arrays to one file in compressed format
savez_compressed('planet_data.npz', X, y)

In [ ]:
import numpy as np
import keras 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from keras.models import Sequential 
from keras.layers import Conv2D,Flatten,MaxPooling2D,Dropout,Dense,BatchNormalization,SpatialDropout2D
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.optimizers import SGD
data = np.load('./planet_data.npz')
X,y =data['arr_0'],data['arr_1']
X

In [ ]:
from keras import backend


def fbeta(y_true, y_pred, beta=2):
    # clip predictions
    y_pred = backend.clip(y_pred, 0, 1)
    # calculate elements
    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
    # calculate precision
    p = tp / (tp + fp + backend.epsilon())
    # calculate recall
    r = tp / (tp + fn + backend.epsilon())
    # calculate fbeta, averaged across each class
    bb = beta ** 2
    fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
    return fbeta_score

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(featurewise_center=True, horizontal_flip=True, vertical_flip=True, rotation_range=90)
test_datagen = ImageDataGenerator(featurewise_center=True)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1)

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

train_datagen.mean = [123.68, 116.779, 103.939]
test_datagen.mean = [123.68, 116.779, 103.939]
train_datagen.fit(X_train)
test_datagen.fit(X_test)

train_it = train_datagen.flow(X_train, Y_train, batch_size=128)

test_it = test_datagen.flow(X_test, Y_test, batch_size=128)

In [ ]:
import numpy as np
import keras 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from keras.models import Sequential 
from keras.layers import Conv2D,Flatten,MaxPooling2D,Dropout,Dense,BatchNormalization,SpatialDropout2D
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.optimizers import SGD

In [ ]:
def build_model():
    in_shape=(64, 64, 3)
    out_shape=17
    
    #load mannequin
    mannequin = VGG16(include_top=False, input_shape=in_shape)

    # mark loaded layers as not trainable

    for layer in mannequin.layers:
        layer.trainable = False

    # enable final vgg block to be trainable

    mannequin.get_layer("block5_conv1").trainable = True

    mannequin.get_layer("block5_conv2").trainable = True

    mannequin.get_layer("block5_conv3").trainable = True

    mannequin.get_layer("block5_pool").trainable = True

    # add new classifier layers

    flat1 = Flatten()(mannequin.layers[-1].output)

    class1 = Dense(128, activation="relu", kernel_initializer="he_uniform")(flat1)

    output = Dense(out_shape, activation="sigmoid")(class1)

    # outline new mannequin

    mannequin = Model(inputs=mannequin.inputs, outputs=output)

    # compile mannequin

    choose = SGD(learning_rate=0.01, momentum=0.9)

    mannequin.compile(optimizer=choose, loss="binary_crossentropy", metrics=[fbeta])
    
    return mannequin

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

# cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
# cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')
save_best_check_point = ModelCheckpoint(filepath='our_model.hdf5', monitor='val_fbeta', mode='max', save_best_only=True, save_weights_only=True)
# checkpoint = ModelCheckpoint(filepath='weights/weights.hdf5',monitor='val_fbeta', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_fbeta', min_delta=0, patience=20, verbose=1, mode='auto')

# fit model
# history = full_model.fit_generator(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)
mannequin.fit_generator(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=10,callbacks=[save_best_check_point])


# evaluate model
loss, fbeta = mannequin.evaluate_generator(test_it, steps=len(test_it), verbose=0)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

In [ ]:
model2 = build_model() # building a sequential model for testing

#loading in the weights of the trained model
model2.load_weights('best_model.hdf5')

In [ ]:
sample_submission = sample_submission_df.copy()
sample_submission['image_name'] = sample_submission['image_name'].apply(lambda x: '{}.jpg'.format(x))
sample_submission.head()

In [ ]:
test1_df = sample_submission.iloc[:40669]['image_name'].reset_index().drop('index', axis=1)
test1_df.head()
test1_df.shape
# initializing an image data generator object for the first 40669 images in the sample submission dataframe
test_image_gen1 = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

# generating the image data for the first 40669 images in the sample submission dataframe
test_gen1 = test_image_gen1.flow_from_dataframe(dataframe=test1_df, directory='https://storage.googleapis.com/kaggle-competitions/kaggle/6322/media/habitation1.jpg', x_col='image_name', y_col=None, batch_size=128, shuffle=False, class_mode=None, target_size=(64,64), validate_filenames=False)

# setting the step size for the testing set for the first 40669 images in the sample submission dataframe
step_test_size1 = int(np.ceil(test_gen1.samples / test_gen1.batch_size))

In [ ]:
test_gen1.reset() # reseting the generator to be sure of avoiding shuffling
pred1 = model2.predict(test_gen1, steps=step_test_size1, verbose=1)

In [ ]:
test_file_names1 = test_gen1.filenames # storing the filenames (images names) of the first 40669 images names in 
                                       # the sample submission dataframe as ordered in the prediction as a 
                                       # variable

# converting the predictions of the first 40669 to tag names
pred_tags1 = pd.DataFrame(pred1)
pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(unique_labels)[x > 0.5]), axis=1)

# converting the predictions of the first 40669 to a dataframe
result1 = pd.DataFrame({'image_name': test_file_names1, 'tags': pred_tags1})
result1.head()

In [ ]:
test2_df = sample_submission.iloc[40669:]['image_name'].reset_index().drop('index', axis=1)
test2_df.head()

In [ ]:
test_image_gen2 = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

# generating the image data for the remaining images in the sample submission dataframe
test_gen2 = test_image_gen2.flow_from_dataframe(dataframe=test2_df, \
            directory='https://storage.googleapis.com/kaggle-competitions/kaggle/6322/media/habitation1.jpg', x_col='image_name', \
            y_col=None, batch_size=128, shuffle=False, class_mode=None, target_size=(64,64), validate_filename=False)

# setting the step size for the testing set for the remaining images in the sample submission dataframe
step_test_size2 = int(np.ceil(test_gen2.samples / test_gen2.batch_size))

In [ ]:
test_gen2.reset() # reseting the generator to be sure of avoiding shuffling
pred2 = model2.predict(test_gen2, steps=step_test_size2, verbose=1)

test_file_names2 = test_gen2.filenames # storing the filenames (images names) of the remaining images names in 
                                       # the sample submission dataframe as ordered in the prediction as a 
                                       # variable

# converting the predictions of the remaining images to tag names
pred_tags2 = pd.DataFrame(pred2)
pred_tags2 = pred_tags2.apply(lambda x: ' '.join(np.array(unique_labels)[x > 0.5]), axis=1)

# converting the predictions of the remaining to a dataframe

len(test_file_names2)
len(pred_tags2)
result2 = pd.DataFrame({'image_name': test_file_names2, 'tags': pred_tags2})
result2.head()

In [ ]:
final_result = pd.concat([result1, result2]) # concatenate the predictions of the test.jpg and 
                                             # test-additional.jpg into a single dataframe

final_result = final_result.reset_index().drop('index', axis=1) # reseting the index of the dataframe so it 
                                                                # matches that of sample submission datafarme

print(final_result.shape)
final_result.head()

assert sum(sample_submission['image_name'] == final_result['image_name']) == 61191
# removing the .jpg extension from 'iamge_name' column
final_result['image_name'] = final_result['image_name'].apply(lambda x: x[:-4])
final_result.head()



In [ ]:
final_result.to_csv('final_submission.csv', index=False) # saving the predictions